# Insurance Pricing Game

## 0. Setup

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [184]:
def get_claims_performance(data, by=None):

    claims_columns = ['claim_amount']

    for column in claims_columns:
        if column not in data.columns:
            raise KeyError(f"Key data missing from dataframe: {column}")
    
    if by is None:
        raise TypeError("Argument not passed to 'by', or None passed instead")

    data_claims = data[data['claim_amount'] > 0]
    
    if by is not None:
        data = data.groupby(by)
        data_claims = data_claims.groupby(by)

    policies = data['id_policy'].count()
    incurred = data['claim_amount'].sum()
    claim_count = data_claims['claim_amount'].count()
    avg_claim_amount = data_claims['claim_amount'].mean().round(2)

    data_grouped = pd.DataFrame({"policies": policies, "incurred": incurred, "claim_count":claim_count, "avg_claim_amount":avg_claim_amount})

    data_grouped['frequency'] = (data_grouped['claim_count'] / data_grouped['policies']).round(2)
    data_grouped['burn'] = (data_grouped['incurred'] / data_grouped['policies']).round(2)

    return data_grouped

In [188]:
def get_policy_demographics(data, by=None):
    def get_normalized_proportion(data, nominal):
        
        unique_values = data[nominal].unique()

        df = pd.DataFrame()

        for value in unique_values:
            df[f"{nominal}_{value}_prop"] = data.groupby(nominal).value_counts(normalize=True)
        
        return df

    def passenger(data, by):
        age_additionaldriver = data.groupby(by)['drv_age2'].mean().round(2)
        age_alldrivers = data[[by,'drv_age1']].append(data[[by, 'drv_age2']].rename(columns={"drv_age2":"drv_age1"})).groupby(by).mean().round(2)

        data_passenger['avg_age_policyholder'] = age_policyholder
        data_passenger['avg_age_additionaldriver'] = age_additionaldriver
        data_passenger['avg_age_alldrivers'] = age_alldrivers

        return data_passenger

    def vehicle(data, by):
        age_vehicle = data.groupby('year')['vh_age'].mean().round(2)

        data_vehicle['avg_age_vh'] = age_vehicle

        return data_vehicle

    pass

## 1. Loading and Preprocessing of Data

In [2]:
raw = pd.read_csv("C:/Users/Dev Work/Documents/aicrowd/insurance/insurance-pricing-game-starter-kit/training.csv")

In [3]:
data = raw.copy()

## 2. Claims Exploration

Important takeaways so far...

1. No data on what the customer paid for their policy.
2. Book is performing better each year, ensure model doesn't do anything to reverse mix changes.
3. Policyholders (all drivers...) are getting older over the years, as are their vehicles. Not capturing young drivers at same rate.
4. pol_usage, pol_payd, vh_make_model, vh_type, vh_value, vh_speed, vh_fuel, vh_weight proportions do not change over years

In [160]:
print(f"Number of Policies: {len(data)}")
print(f"Number of Policies Claim Free: {len(data[data['claim_amount'] == 0])}")
print(f"Number of Policies with Claim: {len(data[data['claim_amount'] != 0])}")
print(f"Overall Frequency: { round(len(data[data['claim_amount'] != 0]) / len(data),3) * 100 }%")
print(f"Total Incurred: EUR { round(sum(data['claim_amount']),2) } ")

burn_crude = sum(data['claim_amount']) / len(data)
print(f"Crude burn cost: EUR { round(burn_crude, 2) } ")

Number of Policies: 228216
Number of Policies Claim Free: 204924
Number of Policies with Claim: 23292
Overall Frequency: 10.2%
Total Incurred: EUR 26057988.08 
Crude burn cost: EUR 114.18 


In [204]:
data_yearly = get_claims_performance(data, 'year')
data_yearly

,policies,incurred,claim_count,avg_claim_amount,frequency,burn
year,,,,,,
1,57054,6806453.23,6280,1083.83,0.11,119.30
2,57054,6788036.71,6030,1125.71,0.11,118.98
3,57054,6404289.79,5747,1114.37,0.10,112.25
4,57054,6059208.35,5235,1157.44,0.09,106.20


In [186]:
data_usage = get_claims_performance(data, 'pol_usage')
data_usage

,policies,incurred,claim_count,avg_claim_amount,frequency,burn
pol_usage,,,,,,
AllTrips,208,72317.31,45,1607.05,0.22,347.68
Professional,16044,2637423.97,2227,1184.29,0.14,164.39
Retired,61988,6875031.45,5836,1178.04,0.09,110.91
WorkPrivate,149976,16473215.35,15184,1084.91,0.10,109.84


In [203]:
x = data.groupby('pol_usage')['vh_type'].value_counts()

for key in x.index:
    print(key)

('AllTrips', 'Tourism')
('AllTrips', 'Commercial')
('Professional', 'Tourism')
('Professional', 'Commercial')
('Retired', 'Tourism')
('Retired', 'Commercial')
('WorkPrivate', 'Tourism')
('WorkPrivate', 'Commercial')


In [205]:
data.groupby('pol_usage')['vh_fuel'].value_counts()

pol_usage     vh_fuel 
AllTrips      Diesel        172
              Gasoline       36
Professional  Diesel      11360
              Gasoline     4640
              Hybrid         44
Retired       Gasoline    32692
              Diesel      29228
              Hybrid         68
WorkPrivate   Diesel      83180
              Gasoline    66704
              Hybrid         92
Name: vh_fuel, dtype: int64

In [208]:
for x in data['pol_usage'].unique():
    print(x)

WorkPrivate
Retired
Professional
AllTrips


In [216]:
def get_normalized_proportion(data, nominal):
        
        unique_values = data[nominal].unique()

        df = pd.DataFrame()

        for value in unique_values:
            df[f"{nominal}_{value}_prop"] = data[nominal].value_counts(normalize=True)
        
        return df

In [217]:
get_normalized_proportion(data, 'pol_usage')

,pol_usage_WorkPrivate_prop,pol_usage_Retired_prop,pol_usage_Professional_prop,pol_usage_AllTrips_prop
WorkPrivate,0.657167,0.657167,0.657167,0.657167
Retired,0.271620,0.271620,0.271620,0.271620
Professional,0.070302,0.070302,0.070302,0.070302
AllTrips,0.000911,0.000911,0.000911,0.000911


In [211]:
x = data.groupby('year')

In [227]:
x['pol_usage'].value_counts(normalize=True) #.unstack('pol_usage')

year  pol_usage   
1     WorkPrivate     0.657167
      Retired         0.271620
      Professional    0.070302
      AllTrips        0.000911
2     WorkPrivate     0.657167
      Retired         0.271620
      Professional    0.070302
      AllTrips        0.000911
3     WorkPrivate     0.657167
      Retired         0.271620
      Professional    0.070302
      AllTrips        0.000911
4     WorkPrivate     0.657167
      Retired         0.271620
      Professional    0.070302
      AllTrips        0.000911
Name: pol_usage, dtype: float64